<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [3]:
%matplotlib inline
%load_ext autoreload
%autoreload 4
%autosave 120

Autosaving every 120 seconds


In [4]:
from fastai.io import *
from fastai.structured import *
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from pandas_summary import DataFrameSummary
from IPython.display import display
from sklearn import metrics
import feather
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import mean_squared_error
import warnings

In [5]:
import gc
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
import time
import matplotlib.gridspec as gridspec
import seaborn as sns
gc.enable()

In [6]:
from sklearn.ensemble import RandomForestRegressor
from boruta import BorutaPy

In [29]:
data = feather.read_dataframe('train_df_beta')

In [30]:
train_missing = (data.isnull().sum() / len(data)).sort_values(ascending = False)

In [31]:
# data.drop(['price_skew', 'price_var', 'price_skew_old','price_var_old'], axis=1, inplace=True)

In [33]:
for df in [data]:
    for f in ['purchase_date_max','purchase_date_min','purchase_date_max_old',\
                     'purchase_date_min_old']:
        df[f] = df[f].astype(np.int64) * 1e-9
        
cols_excluded = ['purchase_date_max', 'purchase_date_max_old', 'card_id', 'first_active_month',
                 'target','outliers','card_id_size', 'card_id_size_old', 
                 'purchase_date_min', 'purchase_date_min_old','first_active_monthYear',
                 'first_active_monthMonth',
                 'first_active_monthWeek',
                 'first_active_monthDay',
                 'first_active_monthDayofweek',
                 'first_active_monthDayofyear',
                 'first_active_monthIs_month_end',
                 'first_active_monthIs_month_start',
                 'first_active_monthIs_quarter_end',
                 'first_active_monthIs_quarter_start',
                 'first_active_monthIs_year_end',
                 'first_active_monthIs_year_start']

In [34]:
train_missing = (data.isnull().sum() / len(data)).sort_values(ascending = False)

In [36]:
train_missing = pd.DataFrame({'fe':train_missing.index, 'm_pct':train_missing.values})

In [37]:
cols_to_dropped = train_missing[train_missing.m_pct>.70]['fe'].values

In [38]:
cols_to_dropped

array(['category_2_2.0_purchase_amount_sum', 'category_2_2.0_purchase_amount_mean',
       'category_2_4.0_purchase_amount_mean', 'category_2_4.0_purchase_amount_sum',
       'category_2_3.0_purchase_amount_sum', 'category_2_3.0_purchase_amount_mean'], dtype=object)

In [39]:
data.drop(cols_to_dropped, axis=1, inplace=True)

In [40]:
data.replace(-np.inf, 0, inplace=True)
data.replace(np.inf, 0, inplace=True)

In [41]:
cols_to_filled = train_missing[train_missing.m_pct>0]['fe'].values

In [42]:
c_to_fill = [c for c in cols_to_filled if c not in cols_to_dropped]

In [43]:
for c in c_to_fill:
    data[c].fillna((data[c].median()), inplace=True)

In [44]:
train_features = [f for f in data if f not in cols_excluded]

In [45]:
target = data['target'].copy()

In [46]:
from sklearn.model_selection import train_test_split

In [105]:
x, x_t, y, y_t = train_test_split(data, data['target'], 
                                 stratify=data['outliers'], test_size=.8, random_state=1)

In [106]:
x, x_t, y, y_t = train_test_split(x, y, 
                                 stratify=x['outliers'], test_size=.2, random_state=1)

In [107]:
(x['outliers']==1).sum(), (x_t['outliers']==1).sum()

(353, 88)

In [108]:
x.shape, x_t.shape

((32306, 247), (8077, 247))

In [109]:
rf = RandomForestRegressor(n_jobs=-1, max_features='auto', max_depth=5)

In [110]:
x.fillna(0, inplace=True)

In [111]:
x.replace(-np.inf, 0, inplace=True)
x.replace(np.inf, 0, inplace=True)

In [112]:
cols_included = ['feature_1','feature_2','feature_3','transactions_count','subsector_id_nunique','merchant_id_nunique','merchant_category_id_nunique','purchase_Month_mean','purchase_Month_min','purchase_Month_max','purchase_Week_nunique','purchase_Week_mean','purchase_Week_min','purchase_Week_max','purchase_Dayofweek_mean','purchase_Dayofweek_min','purchase_Dayofweek_max','purchase_Day_nunique','purchase_Day_mean','purchase_Day_min','purchase_Day_max','purchase_Hour_nunique','purchase_Hour_mean','purchase_Hour_min','purchase_Hour_max','purchase_amount_sum','purchase_amount_max','purchase_amount_min','purchase_amount_mean','purchase_amount_var','purchase_amount_skew','installments_sum','installments_max','installments_mean','installments_var','installments_skew','month_lag_max','month_lag_min','month_lag_mean','month_lag_var','month_lag_skew','month_diff_mean','month_diff_var','month_diff_skew','purchased_on_weekend_mean','category_1_mean','category_2_mean','category_3_mean','card_id_count','price_mean','price_max','price_min','price_var','Christmas_Day_2017_mean','Children_day_2017_mean','Black_Friday_2017_mean','Mothers_Day_2018_mean','duration_mean','duration_min','duration_max','duration_var','duration_skew','amount_month_ratio_mean','amount_month_ratio_min','amount_month_ratio_max','amount_month_ratio_var','amount_month_ratio_skew','category_2_mean_mean','category_3_mean_mean','purchase_date_diff','purchase_date_average','purchase_date_uptonow','purchase_date_uptomin','transactions_count_old','subsector_id_nunique_old','merchant_id_nunique_old','merchant_category_id_nunique_old','purchase_Month_nunique','purchase_Month_mean_old','purchase_Month_min_old','purchase_Month_max_old','purchase_Week_nunique_old','purchase_Week_mean_old','purchase_Week_min_old','purchase_Week_max_old','purchase_Dayofweek_mean_old','purchase_Day_nunique_old','purchase_Day_mean_old','purchase_Day_min_old','purchase_Hour_nunique_old','purchase_Hour_mean_old','purchase_Hour_min_old','purchase_Hour_max_old','purchase_amount_sum_old','purchase_amount_max_old','purchase_amount_min_old','purchase_amount_mean_old','purchase_amount_var_old','purchase_amount_skew_old','installments_sum_old','installments_max_old','installments_mean_old','installments_var_old','installments_skew_old','month_lag_max_old','month_lag_min_old','month_lag_mean_old','month_lag_var_old','month_lag_skew_old','month_diff_max','month_diff_min','month_diff_mean_old','month_diff_var_old','month_diff_skew_old','authorized_flag_mean','purchased_on_weekend_mean_old','category_1_mean_old','category_2_mean_old','category_3_mean_old','card_id_count_old','price_sum','price_mean_old','price_max_old','price_min_old','price_var_old','Christmas_Day_2017_mean_old','Mothers_Day_2017_mean','fathers_day_2017_mean','Children_day_2017_mean_old','Valentine_Day_2017_mean','Black_Friday_2017_mean_old','Mothers_Day_2018_mean_old','duration_mean_old','duration_min_old','duration_max_old','duration_var_old','duration_skew_old','amount_month_ratio_mean_old','amount_month_ratio_min_old','amount_month_ratio_max_old','amount_month_ratio_var_old','amount_month_ratio_skew_old','category_2_mean_mean_old','category_3_mean_mean_old','purchase_date_diff_old','purchase_date_average_old','purchase_date_uptonow_old','purchase_date_uptomin_old','quarter','elapsed_time','days_feature1','days_feature2','days_feature3','days_feature1_ratio','days_feature2_ratio','days_feature3_ratio','feature_sum','feature_mean','feature_max','feature_min','feature_var','card_id_total','card_id_count_total','card_id_count_ratio','purchase_amount_total','purchase_amount_total_mean','purchase_amount_total_max','purchase_amount_total_min','purchase_amount_sum_ratio','hist_first_buy','new_first_buy','hist_last_buy','new_last_buy','month_diff_ratio','installments_total','installments_ratio','price_total','CLV','CLV_old','CLV_ratio']

In [113]:
df_train_columns = [c for c in data.columns.values if c not in cols_excluded]

In [114]:
df_train_columns = [c for c in cols_included if c not in cols_excluded]

In [115]:
len(df_train_columns)

180

In [116]:
%time rf.fit(x[df_train_columns],y)

/home/chandrasekhar/anaconda3/envs/fastai/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


CPU times: user 10.5 s, sys: 28 ms, total: 10.6 s
Wall time: 2.12 s


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=5,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [117]:
rf.score(x[df_train_columns],y)

0.12900347764283293

In [118]:
boruta_feature_selector = BorutaPy(rf, n_estimators='auto', verbose=1, random_state=42, max_iter = 5, perc = 80)

In [119]:
x1 = x[df_train_columns]

In [121]:
gc.collect()

26

In [122]:
%time boruta_feature_selector.fit(x1.values, y)

Iteration: 1 / 5
Iteration: 2 / 5
Iteration: 3 / 5
Iteration: 4 / 5


BorutaPy finished running.

Iteration: 	5 / 5
Confirmed: 	0
Tentative: 	35
Rejected: 	0
CPU times: user 54min 2s, sys: 1.85 s, total: 54min 4s
Wall time: 8min 37s


BorutaPy(alpha=0.05,
     estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=5,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=379, n_jobs=-1,
           oob_score=False,
           random_state=<mtrand.RandomState object at 0x7f649bfc0510>,
           verbose=0, warm_start=False),
     max_iter=5, n_estimators='auto', perc=80,
     random_state=<mtrand.RandomState object at 0x7f649bfc0510>,
     two_step=True, verbose=1)

In [123]:
X_filtered = boruta_feature_selector.transform(x1.values)
X_filtered.shape

(32306, 0)

In [124]:
final_features = list()
indexes = np.where(boruta_feature_selector.support_ == True)

In [125]:
indexes

(array([], dtype=int64),)

In [126]:
for f in np.nditer(indexes):
    final_features.append(df_train_columns[x])
print(final_features)

ValueError: Iteration of zero-sized operands is not enabled

In [ ]:
with open('final_features_boruta.pkl', 'wb') as f:
    pickle.dump(final_features, f)

In [45]:
X1 = data[train_features]

In [26]:
data[data==np.inf] = np.nan
data.fillna(data.mean(), inplace=True)

In [25]:
data.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,target,first_active_monthYear,first_active_monthMonth,first_active_monthWeek,first_active_monthDay,...,days_feature3,days_feature1_ratio,days_feature2_ratio,days_feature3_ratio,feature_sum,feature_prod,feature_mean,feature_max,feature_min,feature_var
0,2017-06-01,C_ID_92a2005557,0.013145,0.008752,0.011428,-0.820283,2017,6,22,1,...,619,0.008078,0.003231,0.001616,8,10,2.666667,5,1,2.081666
1,2017-01-01,C_ID_3d0044924f,0.010712,0.011385,0.010283,0.392913,2017,1,52,1,...,0,0.005195,0.001299,0.000000,5,0,1.666667,4,0,2.081666
2,2016-08-01,C_ID_d639edf6cd,0.010610,0.008752,0.010283,0.688056,2016,8,31,1,...,0,0.002167,0.002167,0.000000,4,0,1.333333,2,0,1.154701
3,2017-09-01,C_ID_186d6a6901,0.010712,0.014166,0.010283,0.142495,2017,9,35,1,...,0,0.007590,0.005693,0.000000,7,0,2.333333,4,0,2.081666
4,2017-11-01,C_ID_cdbd2c0db2,0.008058,0.014166,0.010283,-0.159749,2017,11,44,1,...,0,0.002146,0.006438,0.000000,4,0,1.333333,3,0,1.527525
